#### Импорт необходимых модулей

In [10]:
import sqlite3
import spacy
import pymorphy2
import stanza as stanza
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from collections import Counter

#### Подключение к базе данных

In [11]:
conn = sqlite3.connect('pos.db')
cur = conn.cursor()

#### Получение текста и золотого стадарта из базы данных

В тексте достаточно много слов, с которыми у программы могут возникнуть сложности: преимущественно это морфологические омонимы ("пила": VERB|NOUN, "простой": ADJ|NOUN, "дуло": VERB|NOUN и т.д.).

In [12]:
def clean(text):
    text = text.replace(',', '')
    text = text.replace(':', '')
    text = text.replace('.', '')
    text = text.replace('?', '')
    return text

cur.execute('SELECT sent FROM corpus')
testText = ' '.join([text[0] for text in cur.fetchall()])
text = clean(testText)
print('Text:', testText)

cur.execute('SELECT tags FROM corpus')
golden_standard = ','.join([text[0] for text in cur.fetchall()])
golden_standard = golden_standard.split(',')
print('Manual tagging:', golden_standard)

Text: Год назад я не пила кофе после обеда. Иначе простой или перегон пустых вагонов станут обычным делом. Просто строчка, просто стих, который требовал продолжения, пытаясь всосать в себя пространство, словно удав, чтобы генерировать это самое продолжение. В два часа пополудни ветер внезапно стих, но туча уже закрыла окрестные горы и начала заслонять солнце. Моя посуду, всё думала, почему Мише не повезло, почему он не состоялся как писатель. Мы уже полдня полем сорняки и всё никак не можем закончить. Стволы спиленных деревьев и выкорчеванные мощные пни было решено не уничтожать, а использовать при создании декоративных ландшафтных композиций. Даже не властвуя, женщина Тургенева всегда смела или, по крайней мере, сильна: такова Лиза, такова Елена. Данзас подал Пушкину другой пистолет: дуло первого при падении забилось снегом. Сегодня река голубей, чем была вчера. В казарме на всех стенах висели мечи. За медведем погнался рой пчел, чтобы проучить его. Я бы, наверное, и вправду влепил Ко

#### Стандартизация разметки

Поскольку не все из исследуемых теггеров выделяют краткие формы прилагательных и причастий, а также их сравнительные степени, объединим их одним тегом. Аналогично не все теггеры выделяют инфинитивы глаголов, так что объединим их одним тегом. Остальные теги просто приводятся к общему: например, частицы в pyMorphy обозначаются как PRCL, а в Spacy и Stanza как PART - сделаем их все PRCL.

In [13]:
'''
DEFAULT TAG SET
ADJ, ADVB, NOUN, PRON, PREP, INFN, CONJ, PRCL, GRND, NUM, PRT
'''

def standartize(lst):
    for i in range(len(lst)):
        if lst[i] == 'ADJF' or lst[i] == 'ADJS' or lst[i] == 'COMP' or lst[i] == 'DET':
            lst[i] = 'ADJ'
        elif lst[i] == 'ADV':
            lst[i] = 'ADVB'
        elif lst[i] == 'PART':
            lst[i] = 'PRCL'
        elif lst[i] == 'SCONJ' or lst[i] == 'CCONJ':
            lst[i] = 'CONJ'
        elif lst[i] == 'ADP':
            lst[i] = 'PREP'
        elif lst[i] == 'INFN' or lst[i] == 'AUX':
            lst[i] = 'VERB'
        elif lst[i] == 'PRTS' or lst[i] == 'PRTF':
            lst[i] = 'PRT'
        elif lst[i] == 'NPRO':
            lst[i] = 'PRON'
        elif lst[i] == 'PROPN':
            lst[i] = 'NOUN'
        elif lst[i] == 'NUMR':
            lst[i] = 'NUM'
    return lst

#### Разметка pyMorphy2

In [14]:
def pyMorphyTagging(text):
    morph = pymorphy2.MorphAnalyzer()
    pyMorhy_tags = [str(morph.parse(word)[0].tag.POS) for word in text.split(' ')]
    return standartize(pyMorhy_tags)

pyMorphy_pos = pyMorphyTagging(text)
print('PyMorphy tags:', pyMorphy_pos)
print('PyMorphy accuracy: %.4f' % accuracy_score(pyMorphy_pos, golden_standard))

PyMorphy tags: ['NOUN', 'ADVB', 'PRON', 'PRCL', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADVB', 'ADJ', 'CONJ', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'PRCL', 'NOUN', 'PRCL', 'NOUN', 'ADJ', 'VERB', 'NOUN', 'GRND', 'VERB', 'PREP', 'PRON', 'NOUN', 'CONJ', 'NOUN', 'CONJ', 'VERB', 'PRCL', 'ADJ', 'NOUN', 'PREP', 'NUM', 'NOUN', 'ADVB', 'NOUN', 'ADVB', 'NOUN', 'CONJ', 'NOUN', 'ADVB', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'PRCL', 'VERB', 'ADVB', 'NOUN', 'PRCL', 'VERB', 'ADVB', 'PRON', 'PRCL', 'VERB', 'CONJ', 'NOUN', 'PRON', 'ADVB', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'PRCL', 'ADVB', 'PRCL', 'VERB', 'VERB', 'NOUN', 'PRT', 'NOUN', 'CONJ', 'PRT', 'ADJ', 'NOUN', 'VERB', 'PRT', 'PRCL', 'VERB', 'CONJ', 'VERB', 'PREP', 'NOUN', 'ADJ', 'ADJ', 'NOUN', 'PRCL', 'PRCL', 'GRND', 'NOUN', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'PREP', 'ADJ', 'NOUN', 'ADJ', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'PREP', 'NOUN', 'VERB', 'NOUN', 'ADVB', 'NOUN', 'VER

#### Разметка Stanza

In [15]:
def stanzaTagging(text):
    nlp = stanza.Pipeline('ru', processors='tokenize,pos', verbose=False)
    doc = nlp(text)
    stanza_tags = [str(word.upos) for sentence in doc.sentences for word in sentence.words]
    return standartize(stanza_tags)

stanza_pos = stanzaTagging(text)
print('Stanza tags:', stanza_pos)
print('Stanza accuracy: %.4f' % accuracy_score(stanza_pos, golden_standard))

Stanza tags: ['NOUN', 'ADVB', 'PRON', 'PRCL', 'VERB', 'NOUN', 'PREP', 'NOUN', 'ADVB', 'ADJ', 'CONJ', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'PRCL', 'NOUN', 'PRCL', 'NOUN', 'PRON', 'VERB', 'NOUN', 'VERB', 'VERB', 'PREP', 'PRON', 'NOUN', 'CONJ', 'NOUN', 'CONJ', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'PREP', 'NUM', 'NOUN', 'ADVB', 'NOUN', 'ADVB', 'NOUN', 'CONJ', 'NOUN', 'ADVB', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'PRON', 'VERB', 'ADVB', 'NOUN', 'PRCL', 'VERB', 'ADVB', 'PRON', 'PRCL', 'VERB', 'CONJ', 'NOUN', 'PRON', 'ADVB', 'ADVB', 'NOUN', 'NOUN', 'CONJ', 'PRON', 'ADVB', 'PRCL', 'VERB', 'VERB', 'NOUN', 'VERB', 'NOUN', 'CONJ', 'VERB', 'ADJ', 'NOUN', 'VERB', 'VERB', 'PRCL', 'VERB', 'CONJ', 'VERB', 'PREP', 'NOUN', 'ADJ', 'ADJ', 'NOUN', 'PRCL', 'PRCL', 'VERB', 'NOUN', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'PREP', 'ADJ', 'NOUN', 'ADJ', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'PREP', 'NOUN', 'VERB', 'NOUN', 'ADVB', 'NOUN', 'NO

#### Разметка Spacy

In [16]:
def spacyTagging(text):
    nlp = spacy.load("ru_core_news_sm")
    doc = nlp(text)
    spacy_tags = [str(word.pos_) for sentence in doc.sents for word in sentence]
    return standartize(spacy_tags)

spacy_pos = spacyTagging(text)
print('Spacy tags:', spacy_pos)
print('Spacy accuracy: %.4f' % accuracy_score(spacy_pos, golden_standard))

Spacy tags: ['NOUN', 'ADVB', 'PRON', 'PRCL', 'VERB', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'ADJ', 'CONJ', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'PRCL', 'NOUN', 'PRCL', 'NOUN', 'PRON', 'VERB', 'NOUN', 'VERB', 'VERB', 'PREP', 'PRON', 'NOUN', 'CONJ', 'VERB', 'CONJ', 'VERB', 'PRON', 'ADJ', 'NOUN', 'PREP', 'NUM', 'NOUN', 'NOUN', 'NOUN', 'ADVB', 'NOUN', 'CONJ', 'NOUN', 'ADVB', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'PRON', 'VERB', 'ADVB', 'NOUN', 'PRCL', 'VERB', 'ADVB', 'PRON', 'PRCL', 'VERB', 'CONJ', 'NOUN', 'PRON', 'ADVB', 'ADVB', 'NOUN', 'NOUN', 'CONJ', 'PRON', 'ADVB', 'PRCL', 'VERB', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'CONJ', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', 'PRCL', 'VERB', 'CONJ', 'VERB', 'PREP', 'NOUN', 'ADJ', 'ADJ', 'NOUN', 'PRCL', 'PRCL', 'VERB', 'NOUN', 'NOUN', 'ADVB', 'VERB', 'CONJ', 'ADVB', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'PREP', 'NOUN', 'VERB', 'NOUN', 'ADVB', 'NOUN', 'NOU

#### Выделение n-грамм

Поскольку наиболее точным в разметке из трёх рассмотренных теггеров оказался pyMorphy (91.39%), используем его для выделения n-грамм.

In [17]:
def chunker(bigram, text, pos):
    chunk = [(i, i+len(bigram)) for i in range(len(pos)-len(bigram)+1) if pos[i:i+len(bigram)] == bigram]
    res = [' '.join(text.split(' ')[seq[0]:seq[1]]) for seq in chunk]
    return res

AV_bigram = ['ADVB', 'VERB']
PAN_trigram = ['PREP', 'ADJ', 'NOUN']
PNVN_trigram = ['PREP', 'NOUN', 'VERB']
print('НАР+ГЛ:', chunker(AV_bigram, text, pyMorphy_pos))
print('ПРЕДЛ+ПРИЛ+СУЩ:', chunker(PAN_trigram, text, pyMorphy_pos))
print('ПРЕДЛ+СУЩ+ГЛ:', chunker(PNVN_trigram, text, pyMorphy_pos))

НАР+ГЛ: ['уже закрыла', 'всегда смела', 'вправду влепил']
ПРЕДЛ+ПРИЛ+СУЩ: ['по крайней мере', 'на всех стенах', 'в чаще травы', 'в эту минуту']
ПРЕДЛ+СУЩ+ГЛ: ['при падении забилось', 'За медведем погнался']


#### Улучшение первого ДЗ

Для улучшения качества предсказания из первого домашнего задания предлагаю использовать биграмму "didn't+VERB", тогда сможем однозначно выделять отрицательные отзывы типа "didn't enjoy". В отзывах также часто встречаются оценки вида "the best movie", так что может быть полезным выделять триграммы "the+ADJ(SUPERL)+movie". Очень часто усилительное наречие so встречается в оценках вида "so boring" или "so awesome", причём обычно прилагательным при нём подчеркивается основная линия отзыва, так что можно выделять биграммы "so+ADJ".

In [18]:
conn = sqlite3.connect('reviews.db')
cur = conn.cursor()

cur.execute("SELECT text FROM reviews WHERE polarity='pos'")
goodReviews = [review[0] for review in cur.fetchall()]
cur.execute("SELECT text FROM reviews WHERE polarity='neg'")
badReviews = [review[0] for review in cur.fetchall()]

stop_words = set(stopwords.words('english'))
lemmmatizer=WordNetLemmatizer()

def get_pos(reviews):
    pos = []
    for review in reviews:
        text = nltk.tokenize.word_tokenize(review)
        pos.append(nltk.pos_tag(text))
    return pos

good_pos = get_pos(goodReviews)
bad_pos = get_pos(badReviews)

def get_ngrams(pos):
    so_bigrams = []
    didnt_bigrams = []
    theADJmovie_trigrams = []
    for entry in pos:
        for i, word in enumerate(entry):
            if word[0] == 'so' and entry[i+1][1] == 'JJ':
                so_bigrams.append(' '.join((word[0], entry[i+1][0])))
            elif word[0] == 'did' and entry[i+1][0] == "n't" and entry[i+2][1] == 'VB':
                didnt_bigrams.append(' '.join((word[0]+entry[i+1][0], entry[i+2][0])))
            elif word[0] == 'did' and entry[i+1][0] == 'not' and entry[i+2][1] == 'VB':
                didnt_bigrams.append(' '.join((word[0], entry[i+1][0], entry[i+2][0])))
            elif word[0] == 'the' and entry[i+1][1] == 'JJS' and entry[i+2][0] == 'movie':
                theADJmovie_trigrams.append(' '.join((word[0], entry[i+1][0], entry[i+2][0])))
    return so_bigrams, didnt_bigrams, theADJmovie_trigrams

so_bigrams_good, didnt_bigrams_good, theADJmovie_trigrams_good = get_ngrams(good_pos)
so_bigrams_bad, didnt_bigrams_bad, theADJmovie_trigrams_bad = get_ngrams(bad_pos)

so_bigrams_good_only = set(so_bigrams_good).difference(set(so_bigrams_bad))
so_bigrams_bad_only = set(so_bigrams_bad).difference(set(so_bigrams_good))
didnt_bigrams_good_only = set(didnt_bigrams_good).difference(set(didnt_bigrams_bad))
didnt_bigrams_bad_only = set(didnt_bigrams_bad).difference(set(didnt_bigrams_good))
theADJmovie_trigrams_good_only = set(theADJmovie_trigrams_good).difference(set(theADJmovie_trigrams_bad))
theADJmovie_trigrams_bad_only = set(theADJmovie_trigrams_bad).difference(set(theADJmovie_trigrams_good))

ngrams_good_only = so_bigrams_good_only | didnt_bigrams_good_only | theADJmovie_trigrams_good_only
ngrams_bad_only = so_bigrams_bad_only | didnt_bigrams_bad_only | theADJmovie_trigrams_bad_only

print('N-grams in good reviews only:', ngrams_good_only)
print('N-grams in bad reviews only:', ngrams_bad_only)

cur.execute('SELECT text FROM test')
testReviews = [review[0] for review in cur.fetchall()]

predictedScores = []
for review in testReviews:
    posTest = []
    text = nltk.tokenize.word_tokenize(review)
    posTest.append(nltk.pos_tag(text))
    so_bigrams_test, didnt_bigrams_test, theADJmovie_trigrams_test = get_ngrams(posTest)
    test_ngrams = set(so_bigrams_test) | set(didnt_bigrams_test) | set(theADJmovie_trigrams_test)
    if len(test_ngrams.intersection(ngrams_good_only)) >= len(test_ngrams.intersection(ngrams_bad_only)):
        predictedScores.append('pos')
    else:
        predictedScores.append('neg')

cur.execute('SELECT polarity FROM test')
actualScores = [score[0] for score in cur.fetchall()]

print('Accuracy: %.4f' % accuracy_score(predictedScores, actualScores))

N-grams in good reviews only: {'did not experience', 'so pure', 'so believable', 'the best movie', 'so perfect', 'so amazing', 'so creative', 'did not watch', "didn't get", "didn't watch", 'so fantastic', 'so captivating', 'so talented', "didn't look", 'so ravishing', "didn't need", 'so awesome', 'so incredible', "didn't win", "didn't cry", 'so beautiful', 'so excited', 'the greatest movie', 'so emotional', 'so glad', 'so gripping', "didn't know", 'so impressed', 'so good', 'so deep', 'so intricate', 'so lit', 'did not see', "didn't expect", 'so bittersweet', 'so different', 'so great', 'did not receive', 'so beuautiful', "didn't destroy"}
N-grams in bad reviews only: {"didn't like", 'so awful', 'did not make', "didn't feel", 'so boring', 'so hopeful', "didn't do", 'so stupid', 'did not understand', 'so great.In', "didn't use", 'did not contain', 'so incomprehensible', 'so angry', 'did not mind', 'so smart', "didn't add", 'so dramatic', "didn't make", 'so predictable', 'so blunt', 'so 

Точность предсказания осталась примерно той же: поднялась на 2 процентных пункта. Связано это скорее со слишком малым числом тестовых отзывов (их в целом довольно мало на Марсианина).